# <center><font size="8">Tarifa Dinámica</font></center>

# Introducción al Prototipo de Oferta Dinámica Personalizada

---

## 1. Contexto y Objetivos

El presente proyecto se enfoca en el desarrollo de un sistema avanzado de **Oferta Dinámica (Dynamic Pricing)** con el objetivo primordial de **optimizar los ingresos y la rentabilidad** a través de la **personalización de tarifas**.  

En lugar de utilizar modelos de precio estáticos o ajustes basados únicamente en la demanda global, nuestro prototipo se cimenta en el **análisis individual del consumidor** para determinar el precio óptimo en tiempo real.  

Para lograr esta personalización, hemos diseñado un **pipeline de Machine Learning** que combina técnicas de **econometría**, **aprendizaje no supervisado** y **modelos predictivos**.

---

## 2. Metodología y Arquitectura del Modelo

La propuesta metodológica se basa en **tres fases interconectadas**, que transforman la sensibilidad del cliente en una tarifa personalizada:

---

### A. Estimación y Segmentación por Elasticidad (Fundamento Económico)

El primer pilar del prototipo fue la **estimación rigurosa de la elasticidad precio de la demanda ($\text{EPD}$)**.  
Esta métrica ha sido fundamental para entender el grado de respuesta de la cantidad demandada ante variaciones de precio en nuestros datos históricos.

Posteriormente, aplicamos un **algoritmo de clustering** (*aprendizaje no supervisado*) para **clasificar a los clientes** basándonos en su $\text{EPD}$ y otras características de comportamiento.  

Este *clustering* nos permitió crear **segmentos de clientes bien definidos** (por ejemplo: *“Extremadamente Elásticos”*, *“Inelásticos”*), estableciendo un **rango de ajuste de precios** (aumento o decremento) específico para cada grupo.

---

### B. Clasificación de Clientes Nuevos (Motor de XGBoost)

Una vez establecidos los segmentos de elasticidad, desarrollamos un **clasificador basado en el algoritmo XGBoost (eXtreme Gradient Boosting)**.  

La función de este modelo es crucial en la operativa:  
al recibir los datos de un nuevo cliente o una nueva interacción, **XGBoost predice a cuál de las categorías de elasticidad pre-clasificadas pertenece**.  

Esta predicción es el **factor determinante inicial** para estimar la tarifa personalizada, permitiéndonos saber si el sistema debe apuntar a un **aumento**, un **decremento** o un **precio base**.

---

### C. Determinación de la Oferta Dinámica (Red Neuronal)

Finalmente, para la etapa de determinación del precio, implementamos una **red neuronal artificial**.  

Este modelo de aprendizaje profundo **integra las características de la demanda**, la **clasificación de elasticidad obtenida de XGBoost**, y otras **variables dinámicas** (como inventario, hora del día y actividad de la competencia) para **estimar la oferta dinámica final y personalizada**.  

La red neuronal provee la **flexibilidad** y la **capacidad de capturar relaciones no lineales** necesarias para el ajuste preciso del precio a nivel individual.



# <div align="left"><font size="5">Objetivos</font></div>

## Objetivos del Prototipo de Oferta Dinámica

---

### 1. Estimar la Elasticidad de la Demanda  
Cuantificar la **sensibilidad al precio ($\text{EPD}$)** de los diferentes segmentos de clientes como **fundamento económico** para la toma de decisiones de precio.

---

### 2. Segmentar Clientes por Comportamiento de Precio  
Aplicar **técnicas de *clustering*** para **agrupar clientes** en categorías de elasticidad homogéneas, creando **segmentos accionables** que definan la estrategia inicial de **aumento o decremento de precios**.

---

### 3. Desarrollar un Motor de Clasificación de Clientes  
Implementar y validar el **algoritmo XGBoost** para **clasificar con alta precisión** a nuevos clientes dentro de los segmentos de elasticidad previamente definidos.

---

### 4. Integrar un Pipeline Predictivo Completo  
Crear un prototipo **pipeline robusto** que combine:  
- el *clustering* (**aprendizaje no supervisado**),  
- la **clasificación** (XGBoost), y  
- la **predicción** (Red Neuronal)  

para la **generación de la tarifa final personalizada con tarifa dinámica**.


# <div align="left"><font size="5">Librerias</font></div>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from Tools import Tools4DataExtraction as T4DE
from Tools.Tools4Cluster import ClusteringData
from Tools.Tools4ClasSupervisada import ClusteringSupervisado
from Tools import Tools4TarifPer as T4TP
from Tools import Tools4Net as T4N 
from Tools import Tools4Elasticity as T4E

# <div align="left"><font size="5">Calculo de la elasticidad</font></div>

La Elasticidad Precio de la Demanda (EPD) mide cómo la cantidad demandada de un producto cambia en respuesta a una variación en su precio. Se calcula como 

$$\epsilon_{Q} = \frac{\Delta Q}{\Delta P}$$

### Tipos de Elasticidad de la Demanda

**Demanda Elástica** ($|EPD| > 1$):  
La cantidad demandada varía en una proporción mayor que el cambio en el precio.  
Un pequeño cambio en el precio provoca un cambio significativo en la cantidad demandada.  
*Ejemplo:* Bienes de lujo o productos con muchos sustitutos disponibles.

---

**Demanda Inelástica** ($|EPD| < 1$):  
La cantidad demandada varía en una proporción menor que el cambio en el precio.  
Un cambio en el precio tiene poco impacto en la cantidad demandada.  
*Ejemplo:* Bienes de primera necesidad o esenciales, como medicamentos o gasolina (a corto plazo).

---

**Elasticidad Unitaria** ($|EPD| = 1$):  
La cantidad demandada varía en la misma proporción que el cambio en el precio.


In [2]:
T4E.MainElas()

Memoria usada antes: 623.56 MB
Memoria usada después: 121.20 MB
Reducción: 80.6%
La elasticidad de la demanda es: -0.9054
El precio maximo a vender es: 967.6565


# <div align="left"><font size="5">Extracción de información del modelo de datos</font></div>

In [3]:
# Se extraen los datos del modelo de datos
D4NN, D4C= T4DE.Get_Data()

Memoria usada antes: 623.56 MB
Memoria usada después: 121.20 MB
Reducción: 80.6%


# <div align="left"><font size="5">Clasificación no supervisada</font></div>

### Estrategia de Precios por *Cluster*

---

#### 1. Incrementar Precios (o Reducir Descuentos) 📈  
**Objetivo:** Maximizar el margen en clientes de alto valor.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **2** | Aumentar el precio. | **Élite:** Máximo gasto  y máxima actividad . Demanda inelástica. |
| **0** | Evaluar aumento moderado. | **Alto Valor:** Alto gasto. Monitorear para no perderlos por su menor actividad. |

---

#### 2. Bajar Precios (o Aumentar Promociones) 📉  
**Objetivo:** Impulsar el volumen y reactivar a clientes sensibles al precio o en riesgo.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **5** | Aumentar las promociones. | **Sensibles:** Bajo valor y alta indicación de sensibilidad a promociones. |
| **3** | Ofertas de reactivación. | **En Riesgo:** Gasto inicial alto, pero actividad posterior nula. Necesitan un incentivo fuerte para reengancharse. |

---

#### 3. Mantener Precios (Enfoque en Volumen y Venta Cruzada) 🛍️  
**Objetivo:** Fidelizar y aumentar la compra de productos adicionales sin tocar la base del precio.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **1 y 4** | Mantener precios. | **Muy Activos:** Ya tienen una alta frecuencia de compra. La meta es venderles más productos, no bajarles ni bajarles el precio base. |


In [4]:
ClusteringData(False,D4C)

Modelo K-Means guardado exitosamente como: KmeansAllData.joblib


# 

<div align="left"><font size="5">Extracción de la base de datos de los clusters generados</font></div>

In [5]:
# se extrae la base de datos de los clusters generados
DBClus= T4DE.GetDB()

# <div align="left"><font size="5">Clasificación supervisada</font></div>

In [6]:
# Se aplica la clasificación supervisada a la base de datos de los clusters generados
ClusteringSupervisado(DBClus)

Modelo XGBoost guardado exitosamente como: modelo_xgboost_clientes.json


# <div align="left"><font size="5">Obteniendo la tarifa dinámica</font></div>

In [7]:
TodayDataPD=T4N.ProcessingNet(D4NN)

Epoch 1/2
6918/6918 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 18708.7109 - mae: 47.4258 - mse: 18708.7109 - val_loss: 758.3173 - val_mae: 20.1685 - val_mse: 758.3173
Epoch 2/2
6918/6918 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 451.1250 - mae: 14.2979 - mse: 451.1250 - val_loss: 115.9445 - val_mae: 7.4329 - val_mse: 115.9445
2162/2162 ━━━━━━━━━━━━━━━━━━━━ 1s 517us/step

El Error Absoluto Medio (MAE) final es de: 7.40 [Moneda]

La Raíz del Error Cuadrático Medio (RMSE) final es de: 10.67 [Moneda]
(0, 20)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [8]:
DataElas= T4E.GetDataElasticity()
Elas=DataElas['Elasticidad']

# <div align="left"><font size="5">Obteniendo datos del presente día para la tarifa personalizada</font></div>

In [9]:
# Se obtienen los datos de los clientes para obtener la tarifa personalizada
TodayData4C= T4TP.GetTodayData4Cluster(D4C)

In [10]:
# se simula la obtencion del cluster del cliente que está comprando
# para ver que tarifa personalizada se le asigna

import pandas as pd

# 1. Inicializa una lista vacía para guardar los resultados
lista_resultados = []

# 2. Itera y agrega cada DataFrame a la lista
for row in range(len(TodayData4C)):
    # 1. Crea la fila (DataFrame de 1xN)
    InfoClient = pd.DataFrame(TodayData4C.iloc[row]).T
    
    # 2. Procesa la información (asumo que T4TP.GetCluster devuelve Cluster y desc)
    Cluster, desc = T4TP.GetCluster(InfoClient, DBClus,Elas)
    
    # 3. Agrega la nueva columna
    InfoClient["%_Tarifa Personalizada"] = desc
    InfoClient["Cluster"] = Cluster
    
    # 4. Agrega el DataFrame resultante a la lista
    lista_resultados.append(InfoClient)

# 3. Concatena todos los DataFrames de la lista de una sola vez
TodayDataTP = pd.concat(lista_resultados, ignore_index=True)

# El DataFrame 'final' ahora contiene todas las filas concatenadas

# <div align="left"><font size="5">Merge de las tarifas</font></div>

In [11]:
df_combinado = pd.merge(
    TodayDataPD,
    TodayDataTP,
    on='NOMBRE_PASAJERO',  # La columna clave que comparten
    how='inner'    # El tipo de unión que deseas
)

In [12]:
df_PD= df_combinado[['NOMBRE_PASAJERO', 'EMAIL','Cluster', 'PRECIO DINAMICO', '%_Tarifa Personalizada']]
df_PD ['Precio final con IVA']= df_PD['PRECIO DINAMICO']*(df_PD['%_Tarifa Personalizada']/100)+ df_PD['PRECIO DINAMICO']+df_PD['PRECIO DINAMICO']*0.15
df_PD ['Precio final con IVA']= np.round(df_PD ['Precio final con IVA'])

In [13]:
df_PD

,NOMBRE_PASAJERO,EMAIL,Cluster,PRECIO DINAMICO,%_Tarifa Personalizada,Precio final con IVA
0,DANIEL CEBALLOS RENDON,daniel.ceballos.rendon@ejemplo.com,1,957.755798,0.000000,1101.0
1,JOSE RENTERIA,jose.renteria@ejemplo.com,3,958.100830,-4.527081,1058.0
2,RODOLFO MENDOZA MARTINEZ,rodolfo.mendoza.martinez@ejemplo.com,1,957.538574,0.000000,1101.0
3,MARTIN DURAN,martin.duran@ejemplo.com,1,956.976257,0.000000,1101.0
4,FRANCO RAMOS,franco.ramos@ejemplo.com,1,956.882629,0.000000,1100.0
5,MARTIN PATIÑO,martin.patiño@ejemplo.com,1,956.695129,0.000000,1100.0
6,BENNY VEGA,benny.vega@ejemplo.com,1,956.601379,0.000000,1100.0
7,FILEMON ORTEGA,filemon.ortega@ejemplo.com,1,957.725891,0.000000,1101.0
8,JOSE DE JESUS MEJIA ALCARAS,jose.de.jesus.mejia.alcaras@ejemplo.com,1,956.432556,0.000000,1100.0
9,GONZALO TERRAZAS,gonzalo.terrazas@ejemplo.com,1,956.806824,0.000000,1100.0
